In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy
import os
import glob
import pandas as pd

PATH = ""

In [2]:
PATH = ""
!ls {PATH}

fastai
labeled_text
models
movie-review-sentiment-analysis-kernels-only.ipynb
movie-review-sentiment-analysis-kernels-only-transferred-model.ipynb
movie-review-sentiment-analysis-kernels-only-update1.ipynb
sampleSubmission.csv
test.tsv
TEXT_full.pkl
TEXT_imdb.pkl
TEXT.pkl
tmp
train
train.tsv
valid


In [3]:
df_train = pd.read_csv(f'{PATH}train.tsv', sep="\t")

In [4]:
df_test = pd.read_csv(f'{PATH}test.tsv',sep="\t")

In [8]:
df_test.head(10)

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine
5,156066,8545,intermittently pleasing but
6,156067,8545,intermittently pleasing
7,156068,8545,intermittently
8,156069,8545,pleasing
9,156070,8545,but


In [6]:
df_train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


# Language Model

In [5]:
#drop incomplete sentences in training set
def dropIncomplete (df, label="SentenceId", keepLocation=0):
    sentences = []
    for i in df[label].unique():  
        sentences.append(df[df[label] == i].reset_index(drop=True).loc[keepLocation])
    df_result = pd.DataFrame(sentences)
    return df_result

In [6]:
df_train = dropIncomplete(df_train)

In [7]:
df_test = dropIncomplete(df_test)

In [8]:
df_train = pd.DataFrame(df_train["Phrase"]).reset_index(drop=True)
df_test = pd.DataFrame(df_test["Phrase"]).reset_index(drop=True)

In [9]:
df_train.head()

,Phrase
0,A series of escapades demonstrating the adage ...
1,"This quiet , introspective and entertaining in..."
2,"Even fans of Ismail Merchant 's work , I suspe..."
3,A positively thrilling combination of ethnogra...
4,Aggressive self-glorification and a manipulati...


In [10]:
df_test.head()

,Phrase
0,An intermittently pleasing but mostly routine ...
1,Kidman is really the only thing that 's worth ...
2,Once you get into its rhythm ... the movie bec...
3,I kept wishing I was watching a documentary ab...
4,"Kinnear does n't aim for our sympathy , but ra..."


In [11]:
df_full = df_train.append(df_test).reset_index(drop=True)

In [12]:
df_full[df_full.shape[1]-10:]

,Phrase
11830,"At its worst , it implodes in a series of very..."
11831,Maybe I found the proceedings a little bit too...
11832,Moretti 's compelling anatomy of grief and the...
11833,Montias ... pumps a lot of energy into his nic...
11834,Not sweet enough to liven up its predictable s...
11835,"Nasty , ugly , pointless and depressing , even..."
11836,"With tightly organized efficiency , numerous f..."
11837,They should have called it Gutterball .
11838,"A long-winded , predictable scenario ."


In [13]:
df_full.shape

(11839, 1)

In [17]:
#average number of words in each sentence
np.mean(df_full["Phrase"].apply(lambda x: len(x.split(" "))))

19.047977025086578

In [49]:
spacy_tok = spacy.load('en')

In [50]:
TEXT = data.Field(lower=True, tokenize="spacy")

In [51]:
#bbtt = 70
bs=64; bptt=19

In [52]:
df_train.shape, df_test.shape

((8529, 1), (3310, 1))

In [53]:
md = LanguageModelData.from_dataframes(path=PATH,field=TEXT,col="Phrase",train_df=df_full, val_df=df_test,
                                       test_df=df_test,bs=bs, bptt=bptt)

In [54]:
pickle.dump(TEXT, open(f'{PATH}TEXT_full.pkl','wb'))

In [23]:
ls

fastai@
labeled_text/
models/
movie-review-sentiment-analysis-kernels-only.ipynb
movie-review-sentiment-analysis-kernels-only-transferred-model.ipynb
movie-review-sentiment-analysis-kernels-only-update1.ipynb
sampleSubmission.csv
test.tsv
TEXT_full.pkl
TEXT_imdb.pkl
TEXT.pkl
tmp/
train/
train.tsv
valid/


In [55]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(219, 17902, 1, 268557)

In [56]:
TEXT.vocab.itos[:12]

['<unk>', '<pad>', '<', '>', 'eos', '.', 'the', ',', 'a', 'and', 'of', 'to']

In [57]:
TEXT.vocab.stoi['to']

11

In [58]:
em_sz = 200
nh = 500
nl = 3

In [59]:
opt_fn = partial(optim.Adam, betas = (0.7, 0.99))

In [60]:
learner = md.get_model(opt_fn, em_sz, nh, nl, dropouti=0.05, dropout=0.05,
                       wdrop=0.1, dropoute=0.02, dropouth=0.05)

In [61]:
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [62]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss                              
    0      5.86874    5.733352  
    1      5.510582   5.323128                              
    2      5.295627   5.201433                              
    3      5.168229   4.97615                               
    4      4.932089   4.783189                              
    5      4.769538   4.66728                               
    6      4.731221   4.637032                              
    7      4.808199   4.667988                              
    8      4.614614   4.441589                              
    9      4.488423   4.356996                              
    10     4.338571   4.237494                              
    11     4.175133   4.046654                              
    12     4.087235   3.968333                              
    13     4.02391    3.914336                              
    14     4.037097   3.898286                              



[array([3.89829])]

In [63]:
learner.save_encoder('adam1_movieReview_bptt19_enc')

In [64]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=10)

epoch      trn_loss   val_loss                              
    0      4.241308   4.082727  
    1      4.171602   4.013448                              
    2      4.09656    4.053461                              
    3      3.848913   3.698589                              
    4      3.795498   3.746261                              
    5      3.596768   3.458892                              
    6      3.451824   3.334224                              
    7      3.361029   3.272225                              
    8      3.435255   3.234743                              
    9      3.345696   3.212821                              



[array([3.21282])]

In [65]:
learner.save_encoder('adam3_movieReview_bptt19_10_enc')

In [66]:
learner.load_encoder('adam3_movieReview_bptt19_10_enc')

# Test

In [71]:
#use our torchtext field to numericalize it so we can feed it to our language model.
m = learner.model
ss = """I want to say it wil be"""
s = [TEXT.preprocess(ss)]
t=TEXT.numericalize(s)
' '.join(s[0])

'i want to say it wil be'

In [72]:
#steps to test a language model
# Set batch size to 1
m[0].bs=1
# Turn off dropout
m.eval()
# Reset hidden state
m.reset()
# Get predictions from model
res,*_ = m(t)
# Put the batch size back to what it was
m[0].bs=bs

In [73]:
#Top 10 predictions
nexts = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

['.', 'about', '--', 'a', 'so', ':', 'long', ',', 'in', 'made']

In [74]:
#generate a bit more text
print(ss,"\n")
for i in range(20):
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res, *_ =m(n[0].unsqueeze(0))
print("...")

I want to say it wil be 

. < eos > a movie that 's too busy written and too funny to be funny , but it ...


# Sentiment

In [75]:
#Process training set data with sentiment label
df_trainWLabels = pd.read_csv("train.tsv", sep="\t")
#df_trainWLabels = dropIncomplete(df_trainWLabels)
df_trainWLabels.drop(["PhraseId", "SentenceId"], axis = 1,inplace=True)
df_trainWLabels.reset_index(inplace=True, drop=True)
df_trainWLabels.head()

,Phrase,Sentiment
0,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1
1,A series of escapades demonstrating the adage that what is good for the goose,2
2,A series,2
3,A,2
4,series,2


In [76]:
len(df_trainWLabels)

156060

In [77]:
df_train_suffled = df_trainWLabels.sample(frac=1)

In [78]:
df_train_suffled.head()

,Phrase,Sentiment
119366,too hard to be emotional,2
127268,Once folks started hanging out at the barbershop,2
21882,"... if it had been only half-an-hour long or a TV special , the humor would have been fast and furious -- at ninety minutes , it drags .",1
83765,decides to fight her bully of a husband,2
83789,should pay reparations to viewers .,1


In [50]:
def IntoLabeledFolders(df, folderPath, validRatio=0.3, contentIndex=0, labelIndex=1):
    count = 1
    L = len(df)
    cutPoint = int(round((1-validRatio)*L))

    for index, row in df.iterrows():
        phrase=row[contentIndex]
        label = row[labelIndex]
        if (count<cutPoint):
            filename = os.path.join(folderPath,"train", str(label), str(count))+".txt"
        else:
            filename = os.path.join(folderPath,"valid", str(label), str(count))+".txt"
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename,"w") as f:
            f.write(phrase)
        count = count + 1

In [51]:
IntoLabeledFolders(df=df_train_suffled, folderPath = PATH)

In [79]:
#Show labels ###Start here if already created labeled folders
labelList = !ls {PATH}train
labelList

['0', '1', '2', '3', '4']

In [80]:
#Count number of samples under each label training
labelCount = []
for i in labelList:
    trn_files = !ls {PATH}train/{i}
    labelCount.append(len(trn_files))
labelCount

[4910, 19140, 55597, 23204, 6390]

In [81]:
#Count number of samples under each label in validation
labelCount = []
for i in labelList:
    trn_files = !ls {PATH}valid/{i}
    labelCount.append(len(trn_files))
labelCount

[2162, 8133, 23985, 9723, 2816]

In [82]:
#Review sentences
label = 3
fileIndex = 40
FileList = !ls {PATH}train/{label}
sample = !cat {PATH}train/{label}/{FileList[fileIndex]}
sample

['quite fun in places']

In [83]:
class MultiLabeledMovieReview(torchtext.data.Dataset):
    """Create a MovieReview dataset instance given a path and fields.
    Arguments:
        path: Path to the dataset's highest level directory
        text_field: The field that will be used for text data.
        label_field: The field that will be used for label data.
        Remaining keyword arguments: Passed to the constructor of
            data.Dataset.
    """
    def __init__(self, path, text_field, label_field, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        examples = []
        for label in ['0','1','2','3','4']:
            fnames = glob.glob(os.path.join(path, label, "*.txt"))
            assert fnames, f"can't find file under {path}/{label}"
            for fname in fnames:
                with open(fname, "r") as f: text = f.readline()
                examples.append(data.Example.fromlist([text,label], fields))
        super().__init__(examples, fields, **kwargs)
    
    @staticmethod
    def sort_key(ex): return len(ex.text)
    
    @classmethod
    def splits(cls, text_field, label_field, root='.data', 
               train='train', test='test', **kwargs):
        return super().splits(
            root, text_field=text_field, label_field=label_field,
            train=train, validation=None, test=test, **kwargs)

In [84]:
MovieReviewLabel = data.Field(sequential=False)

In [85]:
TEXT = pickle.load(open(f"{PATH}TEXT_full.pkl","rb")) #open in read and bineary mode

In [86]:
PATH

''

In [87]:
splits = MultiLabeledMovieReview.splits(TEXT, MovieReviewLabel,PATH,train="train", test="valid")

In [88]:
t = splits[0].examples[5]

In [89]:
t.label, " ".join(t.text[:30])

('0', 'worst -- and only -- killer website movie')

In [90]:
bs = 64; bptt = 19

In [91]:
md2 = TextData.from_splits(PATH, splits, bs)

In [92]:
ls

fastai@
labeled_text/
models/
movie-review-sentiment-analysis-kernels-only.ipynb
movie-review-sentiment-analysis-kernels-only-transferred-model.ipynb
movie-review-sentiment-analysis-kernels-only-update1.ipynb
sampleSubmission.csv
test.tsv
TEXT_full.pkl
TEXT_imdb.pkl
TEXT.pkl
tmp/
train/
train.tsv
valid/


In [93]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [94]:
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

In [95]:
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.4, wdrop=0.5, dropoute=0.05, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.load_encoder('adam3_movieReview_bptt19_10_enc')

In [96]:
m3.clip=25.
lrs=np.array([1e-4,1e-4,1e-4,1e-3,1e-2])

In [97]:
# freeze/unfreeze and differential learning rate
m3.freeze_to(-1)
m3.fit(lrs/2, 1, metrics=[accuracy])
m3.unfreeze()
m3.fit(lrs, 1, metrics=[accuracy], cycle_len=1)

epoch      trn_loss   val_loss   accuracy                     
    0      1.225756   1.454782   0.379485  



epoch      trn_loss   val_loss   accuracy                     
    0      1.19055    1.374208   0.410391  



[array([1.37421]), 0.4103907096704536]

In [98]:
m3.fit(lrs, 7, metrics=[accuracy], cycle_len=2, cycle_save_name='MovieReview')

epoch      trn_loss   val_loss   accuracy                     
    0      1.194742   1.355661   0.421916  
    1      1.190895   1.323928   0.436148                     
    2      1.176648   1.313545   0.438876                     
    3      1.133999   1.307074   0.444976                     
    4      1.153755   1.281625   0.46397                      
    5      1.12497    1.286287   0.457966                     
    6      1.102612   1.266341   0.472022                     
    7      1.090571   1.258767   0.473763                     
    8      1.098813   1.238849   0.483113                     
    9      1.061897   1.241319   0.486762                     
    10     1.046057   1.250746   0.484095                     
    11     1.014976   1.212742   0.49873                       
    12     1.016663   1.272033   0.485856                     
    13     0.998064   1.207591   0.503764                      



[array([1.20759]), 0.5037638492083111]

In [99]:
m3.fit(lrs, 3, metrics=[accuracy], cycle_len=10, cycle_save_name='MovieReview')

epoch      trn_loss   val_loss   accuracy                      
    0      1.032701   1.236182   0.515319  
    1      1.026307   1.221975   0.507638                      
    2      0.989347   1.210637   0.504696                      
    3      0.981331   1.179499   0.521223                      
    4      0.974019   1.179538   0.516481                      
    5      0.934968   1.176682   0.528881                      
    6      0.944647   1.153137   0.540309                      
    7      0.939814   1.122244   0.540852                      
    8      0.929467   1.135205   0.543101                      
    9      0.915657   1.126405   0.540739                      
    10     0.954687   1.145194   0.525871                      
    11     0.949187   1.183899   0.516627                      
    12     0.926991   1.158074   0.546305                      
    13     0.917253   1.129904   0.539476                      
    14     0.899947   1.138538   0.548058                   

[array([1.07951]), 0.5734173470388986]

In [100]:
m3.fit(lrs*2, 3, metrics=[accuracy], cycle_len=10, cycle_save_name='MovieReview')

epoch      trn_loss   val_loss   accuracy                      
    0      0.900049   1.31804    0.529656  
    1      0.906962   1.133928   0.563387                      
    2      0.871039   1.124645   0.560782                      
    3      0.872574   1.070637   0.580854                      
    4      0.826138   1.091729   0.57487                       
    5      0.83213    1.092103   0.560083                      
    6      0.799855   1.025597   0.592756                      
    7      0.805425   1.024155   0.591203                      
    8      0.820428   1.016166   0.592013                      
    9      0.779574   1.028961   0.593652                      
    10     0.884673   1.26474    0.5508                        
    11     0.850128   1.083796   0.575192                      
    12     0.82217    1.137643   0.565252                      
    13     0.824535   1.052093   0.591333                      
    14     0.779113   1.044278   0.591102                   

[array([1.05886]), 0.6039281361324342]

# Using a transfered model trained on IMDB data set

In [48]:
TEXT = pickle.load(open("TEXT_imdb.pkl","rb")) #TEXT field for IMDB NLP model
splits = MultiLabeledMovieReview.splits(TEXT, MovieReviewLabel, PATH,train="train", test="valid")

In [49]:
bs = 64; bptt = 70
md2 = TextData.from_splits(PATH, splits, bs)
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

In [50]:
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.4, wdrop=0.5, dropoute=0.05, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.load_encoder('adam3_imdb_10_enc') #NLP model trained on IMDB dataset

In [51]:
m3.clip=25
lrs=np.array([1e-4,1e-4,1e-4,1e-3,1e-2])

In [52]:
# freeze/unfreeze and differential learning rate
m3.freeze_to(-1)
m3.fit(lrs/2, 1, metrics=[accuracy])
m3.unfreeze()
m3.fit(lrs, 1, metrics=[accuracy], cycle_len=1)

epoch      trn_loss   val_loss   accuracy                     
    0      1.139295   1.263814   0.432896  


epoch      trn_loss   val_loss   accuracy                     
    0      1.070088   1.167292   0.488185  


[array([1.16729]), 0.4881846352137183]

In [53]:
m3.fit(lrs, 7, metrics=[accuracy], cycle_len=2, cycle_save_name='MovieReview')

epoch      trn_loss   val_loss   accuracy                     
    0      1.052536   1.141739   0.509442  
    1      1.03386    1.147596   0.511788                      
    2      1.005644   1.210041   0.488842                      
    3      0.961546   1.168524   0.5204                        
    4      0.96673    1.154298   0.534447                      
    5      0.949491   1.172889   0.531779                      
    6      0.956332   1.20941    0.512285                      
    7      0.932446   1.18231    0.534402                      
    8      0.924049   1.2296     0.52465                       
    9      0.901105   1.169018   0.539088                      
    10     0.905614   1.207651   0.517092                      
    11     0.88481    1.211402   0.545079                      
    12     0.885735   1.226406   0.537763                      
    13     0.887049   1.188989   0.545634                      


[array([1.18899]), 0.5456343977969432]

In [54]:
#Not overfitting yet, continue training
m3.fit(lrs, 3, metrics=[accuracy], cycle_len=10, cycle_save_name='MovieReview')

epoch      trn_loss   val_loss   accuracy                      
    0      0.919168   1.194043   0.536609  
    1      0.892308   1.167078   0.558539                      
    2      0.886449   1.177978   0.542236                      
    3      0.889609   1.19618    0.541768                      
    4      0.852477   1.223079   0.535612                      
    5      0.862832   1.18232    0.541867                      
    6      0.84305    1.166159   0.552757                      
    7      0.832242   1.183578   0.551902                      
    8      0.816114   1.201127   0.555013                      
    9      0.833731   1.172674   0.55678                       
    10     0.854878   1.226053   0.541263                      
    11     0.864414   1.158709   0.535323                      
    12     0.832049   1.222211   0.538389                      
    13     0.820791   1.193392   0.545381                      
    14     0.813213   1.188233   0.530849                   

[array([1.18782]), 0.5568040695153538]